In [1]:
import sys
sys.path.append('../compression-filter/src')

import datetime
import os

import dotenv
import xarray

from earth2mip import inference_ensemble, registry
from earth2mip.initial_conditions import cds
from compressed_data_source import CompressedSource, CompressedWrapper
import numpy as np
from zarr_filter import J2KFilter

In [2]:
dlwp_vars = ['t850', 'z1000', 'z700', 'z500', 'z300', 'tcwv', 't2m']
time = datetime.datetime(2018, 1, 1)

In [3]:
dlwp_data_source = cds.DataSource(dlwp_vars)
compressed_source = CompressedWrapper(cds.DataSource(dlwp_vars), J2KFilter((721,1440,1128792064,3,1008981770), libpath='/scratch/lfusco/weather-benchmarks/compression-filter/src/build/lib/libh5z_j2k.so'))

In [4]:
data = dlwp_data_source[time]
compressed_data = compressed_source[time]

Could not reach error target of 0.010000 (370.240845 instead).

In [5]:
data.dtype

dtype('float64')

In [14]:
fig, axs = plt.subplots(2, len(dlwp_vars), figsize=(len(dlwp_vars) * 4, 5))
for i in range(13):
    axs[0, i].imshow(data[i])
    axs[1, i].imshow(compressed_data[i])

array([[[2.47671397e-03, 2.47671397e-03, 2.47671397e-03, ...,
         2.47671397e-03, 2.47671397e-03, 2.47671397e-03],
        [2.90854092e-03, 2.90854092e-03, 2.90465226e-03, ...,
         2.90854092e-03, 2.90854092e-03, 2.90854092e-03],
        [3.13791842e-03, 3.13791842e-03, 3.13403154e-03, ...,
         3.13791842e-03, 3.13791842e-03, 3.13791842e-03],
        ...,
        [1.18830439e-02, 1.31179178e-02, 1.31104388e-02, ...,
         5.12037107e-03, 7.15619543e-03, 9.41290778e-03],
        [9.76688332e-03, 1.40188901e-02, 1.61417163e-02, ...,
         6.64503588e-03, 2.08764975e-03, 3.38557831e-03],
        [5.38424647e-03, 1.37225964e-02, 1.88169699e-02, ...,
         2.37179182e-02, 1.58986445e-02, 6.19796218e-03]],

       [[1.67861479e-03, 6.52321901e-03, 9.54917767e-03, ...,
         1.79458703e-02, 2.23530565e-02, 2.78882745e-02],
        [1.12957231e-02, 6.35534860e-03, 3.58973639e-03, ...,
         2.49274069e-03, 6.98023098e-03, 1.24409183e-02],
        [6.28193592e-03, 